# Main Robot Script

Imports

In [ ]:
# Line following
import lfrobot

# Camera
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

# Model
import numpy as np
import pickle
import time
from IPython.display import clear_output

Initialisation

In [ ]:
# Initialise line following
lfrobot.lfInit()
print('\nLine following initialised')

# Initialise camera
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
print('Camera initialised')

# Load sign detection model
model = pickle.load(open('img_model.p','rb'))
print('Sign detection model loaded')

Operation

In [ ]:
# Without actions

lfrobot.lfStart()
turnSpeed = 0.25 #turn speeds between 0.15 and 0.25 work best
lfrobot.lfTurnSpeed(turnSpeed)

categories=['green','sheep', 'slow', 'stop', 'yellow']

while True:
    ## CAMERA INPUT
    test_img = camera.value
    test_array = test_img.flatten()
    test_array = np.array([test_array])
    
    ## PREDICTION
    prediction = model.predict_proba(test_array)[0]    # prediction vector
    prediction_index = (prediction > 0.75).nonzero()   # indices of high confidence classes

    ## ACTION
    if 1 in prediction_index:     # sheep
        print('sheep')
    elif 0 in prediction_index:   # green
        print('green')
    elif 2 in prediction_index:   # slow
        print('slow')
    elif 3 in prediction_index:   # stop
        print('stop')
    elif 4 in prediction_index:   # yellow
        print('yellow')
    else:                         # free
        print('free')
    
    clear_output(wait=True)
    time.sleep(0.1)


In [ ]:
# With actions

lfrobot.lfStart()
turnSpeed = 0.25 #turn speeds between 0.15 and 0.25 work best
lfrobot.lfTurnSpeed(turnSpeed)

categories=['green','sheep', 'slow', 'stop', 'yellow']
current_speed = 0.1

while True:
    ## CAMERA INPUT
    test_img = camera.value
    test_array = test_img.flatten()
    test_array = np.array([test_array])
    
    ## PREDICTION
    prediction = model.predict_proba(test_array)[0]    # prediction vector
    prediction_index = (prediction > 0.75).nonzero()   # indices of high confidence classes

    ## ACTION
    if 1 in prediction_index:     # sheep
        lfrobot.lfStop()
    elif 0 in prediction_index:   # green
        pass
    elif 2 in prediction_index:   # slow
        current_speed = 0.05
        lfrobot.lfSpeed(current_speed)
    elif 3 in prediction_index:   # stop
        lfrobot.lfStop()
        time.sleep(5)
        current_speed = 0.1
        lfrobot.lfSpeed(current_speed)
        time.sleep(2)
    elif 4 in prediction_index:   # yellow
        lfrobot.lfStop()
    else:                         # free
        lfrobot.lfSpeed(current_speed)
    
    time.sleep_ms(10)


Deinitialise

In [ ]:
lfrobot.lfDeinit()